# Assignment 2

* Implement a linear chain CRF with a library
* Use it on two data sets with the same set of features
* Implement the model and evaluate with F-Score. How are the feature weights which are learnt different between the models?

## File handling and data loading

Loads data from files, for both POS training&testing and NER training&testing files

In [2]:
"""
Notebook Imports
"""
import os
import pycrfsuite # pip install python-crfsuite

In [3]:
class DataLoader:
    """
    Loads input data files and cleans up data into following format:
    
    eg: (Data1, Type1)
    """
    #
    def __init__(self, POS_train_path, POS_test_path, NER_train_path, NER_test_path):
        """
        Loads data files into memory
        """
        __POS_train_path = POS_train_path
        __POS_test_path = POS_test_path
        __NER_train_path = NER_train_path
        __NER_test_path = NER_test_path
        #
        # Loads data from disk into memory
        __POS_train_data, __POS_test_data, __NER_train_data, __NER_test_data = self.__extract_data_from_files(POS_train_path=__POS_train_path,
                                                                                                              POS_test_path=__POS_test_path,
                                                                                                              NER_train_path=__NER_train_path,
                                                                                                              NER_test_path=__NER_test_path)
        #
        # Formats data currently loaded in memory into a formatted structure
        self.__POS_train_structured_data, self.__POS_test_structured_data, self.__NER_train_structured_data,self.__NER_test_structured_data=self.__format_data(POS_train_data=__POS_train_data,
                                                                                                                                                               POS_test_data=__POS_test_data,
                                                                                                                                                               NER_train_data=__NER_train_data,
                                                                                                                                                               NER_test_data=__NER_test_data)
    #
    def __extract_data_from_files(self, POS_train_path, POS_test_path, NER_train_path, NER_test_path):
        """
        Opens data files and returns data in memory
        """
        #
        BASE_DIR = os.path.join( os.path.dirname(os.getcwd()))
        #
        with open(BASE_DIR + POS_train_path) as f:
            POS_train_data = f.read()
        with open(BASE_DIR + POS_test_path) as f:
            POS_test_data = f.read()
        with open(BASE_DIR + NER_train_path) as f:
            NER_train_data = f.read()
        with open(BASE_DIR + NER_test_path) as f:
            NER_test_data = f.read()
        #
        return POS_train_data, POS_test_data, NER_train_data, NER_test_data
    #
    def __format_data(self, POS_train_data, POS_test_data, NER_train_data, NER_test_data):
        """
        Formats data into a python data structure (list of lists)
        eg: (Data1, Type1)
        """
        POS_train_structured_data, POS_test_structured_data, NER_train_structured_data, NER_test_structured_data = [],[],[],[]
        for line in POS_train_data.split("\n"):
            #print(line)
            if line is not None and line != "": 
                sub_list = line.split("\t")
                POS_train_structured_data.append(sub_list)
        for line in POS_test_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t")
                POS_test_structured_data.append(sub_list)
        for line in NER_train_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t|")
                NER_train_structured_data.append(sub_list)
        for line in NER_test_data.split("\n"):
            if line is not None and line != "": 
                sub_list = line.split("\t|")
                NER_test_structured_data.append(sub_list)
        #
        return POS_train_structured_data, POS_test_structured_data, NER_train_structured_data, NER_test_structured_data
    #
    def load_data(self):
        """
        Method wrapper which loads data into memory and returns all relevant training and testing files
        """
        return self.__POS_train_structured_data, self.__POS_test_structured_data, self.__NER_train_structured_data, self.__NER_test_structured_data
#
data_loader_obj = DataLoader(POS_train_path="\\data\\pos\\train.col",
                             POS_test_path="\\data\\pos\\test.col",
                             NER_train_path="\\data\\ner-pol\\train.iob",
                             NER_test_path="\\data\\ner-pol\\test.iob")
POS_train_data, POS_test_data, NER_train_data, NER_test_data = data_loader_obj.load_data()
print("Example: " + str(POS_test_data[0:5]))

Example: [['Measuring', 'NN'], ['cups', 'NNS'], ['may', 'MD'], ['soon', 'RB'], ['be', 'VB']]


## Feature Enhancer

Adds more features to the already loaded vectors, including the following features:

* The word itself, coverted to lower case
* Word Suffix (-2,-3)
* Boolean if string is uppercased
* Boolean if string is a title (eg: Title)
* Boolean if string is a digit
* The postag
* The string before it, converted to lowercase
* Boolean if the string before it is a title (eg: Title)
* Boolean if the string before it is uppercased
* Boolean if the string before it is a digit
* postag of string before it
* The string after it, converted to lowercase
* Boolean if the string after it is a title (eg: Title)
* Boolean if the string after it is uppercased
* Boolean if the string after it is a digit
* postag of string after it

In [4]:
def word2features(doc, i, model_type):
    """
    Accepts a word, and respective POS tag, and converts it into a vector of features
    
    Applies different features to the model, depending on whether we are tackling POS tagging, or NER tagging.
    """
    #print(doc[i])
    word = doc[i][0]
    postag = doc[i][1]
    #
    if model_type == 0: # POS Tagging Features
        #
        features = [
            'bias',
            'word.lower=' + word.lower(),
            'word[-3:]=' + word[-3:],
            'word[-2:]=' + word[-2:],
            'word.isupper=%s' % word.isupper(),
            'word.istitle=%s' % word.istitle(),
            'word.isdigit=%s' % word.isdigit(),
            'postag=' + postag
        ]
        #
        # Preword
        if i > 0:
            word1 = doc[i-1][0]
            postag1 = doc[i-1][1]
            features.extend([
                '-1:word.lower=' + word1.lower(),
                '-1:word.istitle=%s' % word1.istitle(),
                '-1:word.isupper=%s' % word1.isupper(),
                '-1:word.isdigit=%s' % word1.isdigit(),
                '-1:postag=' + postag1
            ])
        else:
            # Beginning of document
            features.append('BOS')
        #
        # Postword
        if i < len(doc)-1:
            word1 = doc[i+1][0]
            postag1 = doc[i+1][1]
            features.extend([
                '+1:word.lower=' + word1.lower(),
                '+1:word.istitle=%s' % word1.istitle(),
                '+1:word.isupper=%s' % word1.isupper(),
                '+1:word.isdigit=%s' % word1.isdigit(),
                '+1:postag=' + postag1
            ])
        else:
            # End of document
            features.append('EOS')
    else:
        #
        features = [
            'bias',
            'word.lower=' + word.lower(),
            'word[-3:]=' + word[-3:],
            'word[-2:]=' + word[-2:],
            'word.isupper=%s' % word.isupper(),
            'word.istitle=%s' % word.istitle(),
            'word.isdigit=%s' % word.isdigit(),
            'postag=' + postag
        ]
        #
        # Preword
        if i > 0:
            word1 = doc[i-1][0]
            postag1 = doc[i-1][1]
            features.extend([
                '-1:word.lower=' + word1.lower(),
                '-1:word.istitle=%s' % word1.istitle(),
                '-1:word.isupper=%s' % word1.isupper(),
                '-1:word.isdigit=%s' % word1.isdigit(),
                '-1:postag=' + postag1
            ])
        else:
            # Beginning of document
            features.append('BOS')
        #
        # Postword
        if i < len(doc)-1:
            word1 = doc[i+1][0]
            postag1 = doc[i+1][1]
            features.extend([
                '+1:word.lower=' + word1.lower(),
                '+1:word.istitle=%s' % word1.istitle(),
                '+1:word.isupper=%s' % word1.isupper(),
                '+1:word.isdigit=%s' % word1.isdigit(),
                '+1:postag=' + postag1
            ])
        else:
            # End of document
            features.append('EOS')
    #    
    return features
#
# Function for extracting features & labels
def extract_features_labels(data, model_type):
    """
    Accepts 2 params:
    
    1) data: upon which to split into training vectors (X) and labels (y)
    2) model_type: upon which to apply specific features
    """
    X, y = [], []
    for i in range(len(data)):
        X.append(word2features(doc=data, i=i, model_type=model_type))
        y.append(data[i][1])
    return X,y

X_POS_train_data, y_POS_train_data = extract_features_labels(data=POS_train_data, model_type=0)
X_POS_test_data,  y_POS_test_data  = extract_features_labels(data=POS_test_data,  model_type=0)
X_NER_train_data, y_NER_train_data = extract_features_labels(data=NER_train_data, model_type=1)
X_NER_test_data,  y_NER_test_data  = extract_features_labels(data=NER_test_data,  model_type=1)
# #
# # A function for extracting features in documents
# def extract_features(doc, model_type):
#     """
#     Accepts 2 params:
    
#     1) data: upon which to split into training vectors (X) and labels (y)
#     2) model_type: upon which to apply specific features
#     """
#     return [word2features(doc, i, model_type) for i in range(len(doc))]
# #
# # A function fo generating the list of labels for each document
# def get_labels(doc):
#     return [label for (token, postag, label) in doc]
# #
# # Model POS
# X_POS_train_data = [extract_features(doc, model_type=0) for doc in POS_train_data]
# y_POS_train_data = [get_labels(doc) for doc in POS_train_data]
# X_POS_test_data = [extract_features(doc, model_type=0) for doc in POS_test_data]
# y_POS_test_data = [get_labels(doc) for doc in POS_test_data]
# #
# # Model NER
# X_NER_train_data = [extract_features(doc, model_type=1) for doc in NER_train_data]
# y_NER_train_data = [get_labels(doc) for doc in NER_train_data]
# X_NER_test_data = [extract_features(doc, model_type=1) for doc in NER_test_data]
# y_NER_test_data = [get_labels(doc) for doc in NER_test_data]
#
print("X POS Train Vector: \n" + str(X_POS_train_data[0:5]) + "\n")
print("y POS Train Labels: \n" + str(y_POS_train_data[0:5]) + "\n")
print("X POS Test Vector: \n" + str(X_POS_test_data[0:5]) + "\n")
print("y POS Test Labels: \n" + str(y_POS_test_data[0:5]) + "\n")
print("X NER Train Vector: \n" + str(X_NER_train_data[0:5]) + "\n")
print("y NER Train Labels: \n" + str(y_NER_train_data[0:5]) + "\n")
print("X NER Test Vector: \n" + str(X_NER_test_data[0:5]) + "\n")
print("y NER Test Labels: \n" + str(y_NER_test_data[0:5]) + "\n")


X POS Train Vector: 
[['bias', 'word.lower=in', 'word[-3:]=In', 'word[-2:]=In', 'word.isupper=False', 'word.istitle=True', 'word.isdigit=False', 'postag=IN', 'BOS', '+1:word.lower=an', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=DT'], ['bias', 'word.lower=an', 'word[-3:]=an', 'word[-2:]=an', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=DT', '-1:word.lower=in', '-1:word.istitle=True', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=IN', '+1:word.lower=oct.', '+1:word.istitle=True', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNP'], ['bias', 'word.lower=oct.', 'word[-3:]=ct.', 'word[-2:]=t.', 'word.isupper=False', 'word.istitle=True', 'word.isdigit=False', 'postag=NNP', '-1:word.lower=an', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=DT', '+1:word.lower=19', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=True', '+1:postag=CD'], 

## Training Model - POS

In [5]:
trainer_POS = pycrfsuite.Trainer(verbose=True)
#
# Submit training data to the trainer
for xseq, yseq in zip(X_POS_train_data, y_POS_train_data):
    #print(xseq)
    #print(yseq)
    trainer_POS.append([xseq], [yseq])
# print(len(X_POS_train_data))
# print(len(y_POS_train_data))
# trainer_POS.append(X_POS_train_data, y_POS_train_data)
#
# Set the parameters of the model
trainer_POS.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 100,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})
#
# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer_POS.train('POS_crf.model')
print('POS Model Trained..')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 262822
Seconds required: 4.348

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 2360586.971902
Feature norm: 1.000000
Error norm: 382654.045461
Active features: 257115
Line search trials: 1
Line search step: 0.000002
Seconds required for this iteration: 14.225

***** Iteration #2 *****
Loss: 1790415.127315
Feature norm: 3.984647
Error norm: 395183.006906
Active features: 249101
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.172

***** Iteration #3 *****
Loss: 1492928.087256
Feature norm: 5.501346
Error norm: 326408.670839
Active features: 254233
Line search trials: 1
Line search step: 1.000000
Second

## Training Model - NER

In [6]:
trainer_NER = pycrfsuite.Trainer(verbose=True)
#
# Submit training data to the trainer
for xseq, yseq in zip(X_NER_train_data, y_NER_train_data):
    #print(xseq)
    #print(yseq)
    trainer_NER.append([xseq], [yseq])
# print(len(X_POS_train_data))
# print(len(y_POS_train_data))
# trainer_POS.append(X_POS_train_data, y_POS_train_data)
#
# Set the parameters of the model
trainer_NER.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 100,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})
#
# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer_NER.train('NER_crf.model')
print('NER Model Trained..')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 87931
Seconds required: 0.936

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 145462.707507
Feature norm: 1.000000
Error norm: 63866.017636
Active features: 87548
Line search trials: 1
Line search step: 0.000002
Seconds required for this iteration: 0.528

***** Iteration #2 *****
Loss: 138685.517621
Feature norm: 1.113642
Error norm: 45870.509869
Active features: 84035
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.258

***** Iteration #3 *****
Loss: 125876.600159
Feature norm: 1.321173
Error norm: 45152.827494
Active features: 84015
Line search trials: 1
Line search step: 1.000000
Seconds required 

## Evaluation